In [4]:
import requests
from pprint import pprint

In [424]:
url = 'http://localhost:5006/webhooks/callassist/webhook'
data = {
  "sender": "tsc689888us8gd8x",  
  # "message": "servicer：好的"
  # "message": "servicer：单号是多少"
  # "message": "你好，我的快递发货了吗"
  # "message": "你好，我的快递到哪了"
  # "message": "这个寄的是什么,中通比你们好多了"
  # "message": "单号是89012"
  "message": "30块"
  # "message": "yt896"
  # "message": "九零一二三四,我是收件人"
  # "message": "五六七八九零一二三四"
  # "message": "53698 yt 2515619553698"
  # ,"metadata" :{'express_id':"YT1234567890125"}
}
res = requests.post(url, json=data)
pprint(res)
# pprint(res.text)
d = res.json()
pprint(d)

<Response [200]>
[{'recipient_id': 'tsc689888us8gd8x', 'text': '请继续说您的运单号'},
 {'recipient_id': 'tsc689888us8gd8x',
  'text': '您好，麻烦您提供一下YT+13位数的圆通运单号码，我帮您查看一下'},
 {'api_exception': 0,
  'last_message': {'confidence': 0.45,
                   'exact_hit': False,
                   'input_channel': 'callassist',
                   'intent_name': 'is_ok',
                   'slots': {},
                   'text': '30块',
                   'top2_diff': 0.3731},
  'recipient_id': 'tsc689888us8gd8x',
  'story': 'faq'}]


In [ ]:
# url = 'http://10.130.118.125:5005/webhooks/wdgj/webhook'
url = 'http://10.130.118.125:5006/webhooks/callassist/webhook'
data = {
  "sender": "tst1c44gg056465",  
  # "message": "servicer：好的"
  # "message": "servicer：单号是多少"
  # "message": "我快递到哪里了"
  # "message": "呃这个单号8960357027501"
  # "message": "呃如果是一个件退回，可能是有问题，如果两个件都退回了的话，我们需要帮您去核实到底是哪里出了问题，尽快给您处理，给您回电啊。"
  # "message": "1234"
  "message": "53698 yt 2515619553698"
  # "message": "呃这个单号8960357027501"
  # ,"metadata" :{'express_id':"YT1234567890123"}
}
res = requests.post(url, json=data)
# pprint(res)
# pprint(res.text)
d = res.json()
pprint(d)

[{'recipient_id': 'tst1c44gg056465', 'text': '您好，这边看到您提供的单号不对'},
 {'recipient_id': 'tst1c44gg056465', 'text': '运单号：yt2515619553698'},
 {'recipient_id': 'tst1c44gg056465', 'text': '非常抱歉，给您带来不好的体验了'},
 {'recipient_id': 'tst1c44gg056465',
  'text': '亲，非常抱歉，查询到您的包裹没有物流信息更新，辛苦您提供一下您的姓名和联系方式，小圆马上联系网点查询包裹状态。有结果尽快反馈您请您不要着急。'},
 {'api_exception': 0,
  'last_message': {'confidence': 0.9475,
                   'exact_hit': False,
                   'input_channel': 'callassist',
                   'intent_name': 'inform',
                   'slots': {'slot_express_id': 'yt2515619553698'},
                   'text': '53698yt2515619553698',
                   'top2_diff': 0.9416},
  'recipient_id': 'tst1c44gg056465',
  'story': 'check_express_status'}]


In [2]:
import pandas as pd
import re

weihao_pat = re.compile(r"客服[^客]{,50}尾[号数].{,3}\d{3,6}")
weihao_pat1 = re.compile(r"尾[号数].{,3}\d{3,6}")
yt_pat = re.compile(r"[yY].{,2}[tT]")
# 读取Excel文件
df = pd.read_excel('D:\客服AI质检项目\转写拼接2024-03-22dev.xlsx', header=0)
url = 'http://10.130.118.125:5006/webhooks/callassist/webhook'
data = {
  "sender": "tst1c44gg056465",  
  "message": "1234"
}

fout_path = 'D:\客服AI质检项目\order_test0407.txt'
# 显示前几行数据
print(df.index)
df.iloc[1]

RangeIndex(start=0, stop=1565, step=1)


sessionId                                     1710980202.10119
content      客户：你好，我昨天我查了一个快递，他说的是通风南路37 号，我看这边没找到圆你们圆通的点嘛\...
caller                                           18344703509.0
waybillNo                                                  NaN
Name: 1, dtype: object

In [5]:
sum = 0
miss = 0
with open(fout_path, 'w') as fout:
    for i in df.index:
        get_express_id = False
        get_phone = False
        order_dt = df.iloc[i]
        sesid = order_dt['sessionId']
        content = order_dt['content']
        # if weihao_pat1.search(content) and not weihao_pat.search(content):
        if yt_pat.search(content):
            sum += 1
            res_ls = []
            content_ls = order_dt['content'].split('\n')
            for txt in content_ls:
                if not txt: continue
                if txt.startswith('客户'):
                    data = {"sender": sesid, "message": txt.replace("客户：", '')}
                else:
                    data = {"sender": sesid, "message": txt.replace("客服", 'servicer')}
                res = requests.post(url, json=data)
                d = res.json()
                if d and d[-1]['last_message'].get('intent_name') == 'predict_call_end':
                    print(d[-1]['last_message'])
                continue
                res_ls.append(d)
                # print(txt)
                if 'slot_express_id' in d[-1]['last_message'].get('slots'):
                    # 'slot_express_id'
                    get_express_id = True
                if 'slot_phone' in d[-1]['last_message'].get('slots'):
                    # 'slot_phone'
                    get_phone = True
            continue
            if not get_express_id or get_phone:
                miss += 1
                print(f'number: {i},{sesid},{sum}, {miss}')
                fout.write(f'number: {i},{sesid},{sum}, {miss}\n')
                print(content)
                fout.write(f"{content}\n")
                pprint(res_ls)
                fout.write(f"{res_ls}\n")
            if miss >= 10:
                break

{'text': '模型文本是。对我们这边更往哪些去核实一下，对，这是模型文本', 'input_channel': 'callassist', 'intent_name': 'predict_call_end', 'confidence': 0.2746, 'top2_diff': 0.061, 'exact_hit': False, 'slots': {}}
{'text': '模型文本是。嗯，好的，请问还有其他问题吗，这是模型文本', 'input_channel': 'callassist', 'intent_name': 'predict_call_end', 'confidence': 0.3576, 'top2_diff': 0.2148, 'exact_hit': False, 'slots': {}}
{'text': '模型文本是。好的好的，那您这边还有其他的问题吗，这是模型文本', 'input_channel': 'callassist', 'intent_name': 'predict_call_end', 'confidence': 0.4121, 'top2_diff': 0.2438, 'exact_hit': False, 'slots': {}}
{'text': '模型文本是。好的，那您这边还有其他需要咨询的吗？女士，这是模型文本', 'input_channel': 'callassist', 'intent_name': 'predict_call_end', 'confidence': 0.3722, 'top2_diff': 0.1777, 'exact_hit': False, 'slots': {}}
{'text': '模型文本是？给您添麻烦。抱歉，女士，这个先目目前显示您看成都转运中心具体在里面是需要注意专家去哪些当地网点核实之后给您回复，这是模型文本', 'input_channel': 'callassist', 'intent_name': 'predict_call_end', 'confidence': 0.166, 'top2_diff': 0.0382, 'exact_hit': False, 'slots': {}}
{'text': '模型文本是还有什么想要咨询吗，这是模型文本', 'input

KeyboardInterrupt: 

In [7]:
sum = 0
miss = 0
with open(fout_path, 'w') as fout:
    for i in df.index:
        get_express_id = False
        get_phone = False
        order_dt = df.iloc[i]
        sesid = order_dt['sessionId']
        content = order_dt['content']
        # if weihao_pat1.search(content) and not weihao_pat.search(content):
        if yt_pat.search(content):
            sum += 1
            res_ls = []
            content_ls = order_dt['content'].split('\n')
            for txt in content_ls:
                if not txt: continue
                if txt.startswith('客户'):
                    data = {"sender": sesid, "message": txt.replace("客户：", '')}
                else:
                    data = {"sender": sesid, "message": txt.replace("客服", 'servicer')}
                res = requests.post(url, json=data)
                d = res.json()
                res_ls.append(d)
                # print(txt)
                if 'slot_express_id' in d[-1]['last_message'].get('slots'):
                    # 'slot_express_id'
                    get_express_id = True
                if 'slot_phone' in d[-1]['last_message'].get('slots'):
                    # 'slot_phone'
                    get_phone = True
            if not get_express_id or get_phone:
                miss += 1
                print(f'number: {i},{sesid},{sum}, {miss}')
                fout.write(f'number: {i},{sesid},{sum}, {miss}\n')
                print(content)
                fout.write(f"{content}\n")
                pprint(res_ls)
                fout.write(f"{res_ls}\n")
            if miss >= 10:
                break

number: 3,1710980964.10119,1, 1
客服：你好，很高兴为您服务，请问有什么可以帮您
客户：嗯，我想问一下，我这边呢有一个件了，然后是到集运仓中转中通快递的。然后他那边理赔说理赔到你们这个圆通，我们第一单物流是圆通发出的嘛
客服：嗯
客服：嗯
客服：，你现在这个东
客户：，他那我我问一下这个怎么你们那边有收到吗？那个圆通的那个
客户：就中通的理赔件了
客服：，您说的是哪个单号呢？女士，请您提供一下圆通的单号呢？女士
客户：啊
客户：，好，yt8953 
客服：8953 ，然后呢
客户：5158
客服：嗯
客户：59768
客服：浙江台州寄出，寄往陕西省西安市，是吗
客户：嗯，对
客服：？这是一月份的件是吗
客户：对
客服：？嗯
客户：，这个是
客户：01月28日的件，然后是02月05日，你们到中转仓的
客户：，然后中转仓，再从转寄的话是中通快递，然后那边丢件了，他说理赔到你们圆通公司的
客服：，请问您贵姓是怎么称呼您女士？您是发件人吗？还是后来的
客户：啊，我姓刘
客户：啊
客服：您是发件人吗？还是收货人呢
客户：，我是发起人
客服：？这是邮寄的什么物品呢？女士方便提供一下吗
客户：嗯，垃圾桶
客服：？垃圾桶是吗
客户：。对
客服：？多少钱呢？大家真女士
客户：嗯，81 
客服：就这个件的话，是到了中转仓，中转仓那边拒收退回的吗
客户：不是拒收退回的，是他重新转发到新疆的吗？这个不是
客服：？嗯
客户：，然后在路途中转新疆的路途中丢件了
客服：，那就是没有传寄到新疆，是吗
客户：。对，不是转寄到新疆了，已经到新疆了，然后转再发往收货人地那个路中丢失的
客服：？要的物流
客服：，那您这个件我们稍后跟王里去核实之后，给您回电处理一下。因为他这边第一段物流显示是已经签收了，您说是呃，第2个物流
客户：就到乌鲁木齐嘛。那边还在乌鲁木齐的时候，嗯，乌鲁木齐转运中心已经丢件了
客户：。嗯，好的
客户：，对，他们是他们那边说理赔
客服：，但是我这边没有查询到网点，这边是上报理赔的情况
客户：，理赔到你们公司的
客户：这样的了哦，好，那你帮我问一下吧
客服：。对我们这边更往哪些去核实一下，对
客户：啊，好嗯，再见
客服：您方便留下手机号码吗？女士
客户：嗯
客服：198 ，然后呢
客户：，198 
客户：576
客服：，然后呢
客户：52536
客服：，1

In [88]:
sum = 0
miss = 0
with open(fout_path, 'w') as fout:
    for i in df.index:
        get_express_id = False
        order_dt = df.iloc[i]
        sesid = order_dt['sessionId']
        content = order_dt['content']
        # if weihao_pat1.search(content) and not weihao_pat.search(content):
        if re.search(r'手机|电话', content) and re.search(r'(?<![\dyt])1\d{2,}', content):
            print(content)
            # sum += 1
            # res_ls = []
            # content_ls = order_dt['content'].split('\n')
            # for txt in content_ls:
            #     if txt.startswith('客户'):
            #         data = {"sender": sesid, "message": txt.replace("客户：", '')}
            #         res = requests.post(url, json=data)
            #         d = res.json()
            #         res_ls.append(d)
            #         if 'slot_express_id' in d[-1]['last_message'].get('slots'):
            #             # 'slot_express_id'
            #             get_express_id = True
            #             break
            # if not get_express_id:
            #     miss += 1
            #     print(f'number: {i},{sesid},{sum}, {miss}')
            #     fout.write(f'number: {i},{sesid},{sum}, {miss}\n')
            #     print(content)
            #     fout.write(f"{content}\n")
            #     pprint(res_ls)
            #     fout.write(f"{res_ls}\n")
            # break

客户：呃，那个呃我需要那个修改一下那个地址，那个寄件那个收件的地址
客服：1590 服务，请问什么帮您？您好
客服：，那您这边是这个包裹的寄件人还是收件人呢
客户：，我是收件人啊
客服：？收件我给您查询了一下，您的是发件，那边有商贸给您根治
客户：，他他给我更改了吗
客服：。嗯，对，这个建议的话，它是上报一个拦截根止有效的
客户：？呃，他是跟到温州了是吗
客服：跟到大概的一个省区的话，是浙江温州那边，但后面的话它隐藏了，我看不到
客户：？呃，可以可以，那为什么我这边看还是在宁波呢？我之前打电话问过那个
客服：，因为他他这个件需要到达宁波那边，做下车以后才能给你转出。因为他这个件正在发往宁波的这个路上，他不可能说半路就给你转过去啊，先生
客户：哎哎哎，好嘞，谢谢啊
客服：，好的，已经修改了，您放心。先生，可能是因为修改地址，它这边运输时效可能有些延缓，其他都没有问题
客户：，嗯好
客户：好的好的嗯
客服：。好的，那您还有其他需要咨询的吗
客户：，没有没有没有没有，谢谢啊嗯

客服：你好，很高兴为您服务，请问有什么可以帮您
客户：嗯，我想问一下，我这边呢有一个件了，然后是到集运仓中转中通快递的。然后他那边理赔说理赔到你们这个圆通，我们第一单物流是圆通发出的嘛
客服：嗯
客服：嗯
客服：，你现在这个东
客户：，他那我我问一下这个怎么你们那边有收到吗？那个圆通的那个
客户：就中通的理赔件了
客服：，您说的是哪个单号呢？女士，请您提供一下圆通的单号呢？女士
客户：啊
客户：，好，yt8953 
客服：8953 ，然后呢
客户：5158
客服：嗯
客户：59768
客服：浙江台州寄出，寄往陕西省西安市，是吗
客户：嗯，对
客服：？这是一月份的件是吗
客户：对
客服：？嗯
客户：，这个是
客户：01月28日的件，然后是02月05日，你们到中转仓的
客户：，然后中转仓，再从转寄的话是中通快递，然后那边丢件了，他说理赔到你们圆通公司的
客服：，请问您贵姓是怎么称呼您女士？您是发件人吗？还是后来的
客户：啊，我姓刘
客户：啊
客服：您是发件人吗？还是收货人呢
客户：，我是发起人
客服：？这是邮寄的什么物品呢？女士方便提供一下吗
客户：嗯，垃圾桶
客服：？垃圾桶是吗
客户：。对
客服：？多少钱呢？大家真女士
客户：嗯，81 
客服：就这个件的话，是到了中转仓，中转仓那边拒收退回的吗
客户

<!-- 对某个工单单独运行分析 -->

In [14]:
sum = 0
miss = 0
url = 'http://10.130.118.125:5005/webhooks/callassist/webhook'

# with open(fout_path, 'w') as fout:
if True:
    for i in df.index:
        get_express_id = False
        get_phone = False
        order_dt = df.iloc[i]
        sesid = str(order_dt['sessionId'])
        # print(type(sesid))
        content = order_dt['content']
        # if weihao_pat1.search(content) and not weihao_pat.search(content):
        if sesid == '1710982444.10119':
            sum += 1
            res_ls = []
            content_ls = order_dt['content'].split('\n')
            for txt in content_ls:
                if not txt: continue
                if txt.startswith('客户'):
                    data = {"sender": sesid, "message": txt.replace("客户：", '')}
                else:
                    data = {"sender": sesid, "message": txt.replace("客服", 'servicer')}
                res = requests.post(url, json=data)
                d = res.json()
                res_ls.append(d)
                # print(txt)
                if 'slot_express_id' in d[-1]['last_message'].get('slots'):
                    # 'slot_express_id'
                    get_express_id = True
                if 'slot_phone' in d[-1]['last_message'].get('slots'):
                    # 'slot_phone'
                    get_phone = True
                print(f'number: {i},{sesid},{sum}, {miss}')
                # fout.write(f'number: {i},{sesid},{sum}, {miss}\n')
                print(content)
                # fout.write(f"{content}\n")
                pprint(res_ls)
                # fout.write(f"{res_ls}\n")
            break

number: 12,1710982444.10119,1, 0
客服：您好，很高兴为您服务，请问有什么可以帮您
客户：对吧
客服：？你好，女士
客户：？喂，你好，能不能帮我们把快递揽收一下
客服：，请问您贵姓怎么称呼您女士
客户：弄王
客服：，您是要寄件吗？还是插件呢
客户：用我啊
客服：？好的女士，您是下单了吗
客户：。嗯是的
客服：？是您这个来电的手机号码下的订单吗
客户：，不是不是
客服：？那是哪个号码下的订单呢？麻烦您提供一下手机号码呢，女士
客户：18979
客服：8000
客户：，不用不用不用
客户：570
客服：890570 ，然后呢
客户：，你上面是有什么吗
客服：，好的
客户：？稍等一下哈，我看一下
客户：那个189 
客服：好的
客服：，嗯
客户：9347
客服：，189 
客户：570
客服：93470570 是吧
客户：。嗯，是的
客服：，18993470570 
客户：嗯，是
客服：，是这个号码下的订单吗
客户：是的
客服：？您在平台上下的订单吗
客户：是的
客服：？但是没有查询到订单号女士
客户：，我我们是怎么样
客服：，好的，您再看一下呢
客户：？稍等一下，我再看一下哈
客户：啊，我报一下订单号，你你查一下吧
客服：，请说女士
客户：，yt
客服：嗯
客户：7452
客服：，然后呢
客户：7702
客服：，然后
客户：93661
客服：这个单号没有任何信息女士
客户：没有信息吗？我这上面显示包裹正在揽收啊
客服：，对
客服：，没有，他这个件没有交付，圆通公司揽收，没有任何信息女士
客户：，那我我我
客服：，他这个件发件人还没给到网点公司揽收，没有查询到任何信息女士
客户：嗯那好吧
客服：。嗯，好的，请问还有其他问题吗
客户：，没了没了

[[{'recipient_id': '1710982444.10119', 'text': '标准话术执行率：90%'},
  {'recipient_id': '1710982444.10119', 'text': '违规话术识别：无违规话术'},
  {'api_exception': 0,
   'last_message': {'confidence': 0.583,
                    'exact_hit': False,
                    'i

In [35]:
import re
repat_numbers = re.compile(r"[零令林一幺妖二两三四五六七八九]{3,}")

In [86]:
mt = repat_numbers.search("二三九")
print(mt.group())

二三九


In [ ]:
repat_numbers = re.compile("\\d{13}(?!\\d)")
mt = repat_numbers.search("12345678901234")
print(mt)

In [57]:
print(re.search('q\s+w', 'dddq  wer'))
express_id_pat4 = re.compile("(?<!\\d)\\d{13}(?!\\d)")
express_id_pat4.search('1234567890123')

<re.Match object; span=(3, 7), match='q  w'>


<re.Match object; span=(0, 13), match='1234567890123'>

''

In [2]:
import jieba
import jieba.posseg as pseg

In [88]:
pseg.lcut('0666')

[pair('0666', 'm')]

In [77]:
tem_ls = [[{'recipient_id': '1710982409.10119', 'text': '您好，麻烦您提供一下10、12或YT+13位数的圆通运单号码，我帮您看一下。'}, {'recipient_id': '1710982409.10119', 'last_message': {'text': '啊，我在你们圆通好几个快递都给我算成放到快递柜儿了。然后我不允许放快递柜，就要上门送到送上门', 'input_channel': 'callassist', 'intent_name': 'home_delivery', 'confidence': 0.9349, 'top2_diff': 0.9232, 'exact_hit': False, 'slots': {}}, 'story': 'faq', 'api_exception': 0}], [{'recipient_id': '1710982409.10119', 'text': '您好，麻烦您提供一下YT+13位数的圆通运单号码，我帮您查看一下'}, {'recipient_id': '1710982409.10119', 'text': '您好，麻烦您提供一下10、12或YT+13位数的圆通运单号码，我帮您看一下。'}, {'recipient_id': '1710982409.10119', 'last_message': {'text': '快好几个快递呢。我我我我都要告诉你们，好几个快递都给我放空着', 'input_channel': 'callassist', 'intent_name': 'package_lost', 'confidence': 0.7744, 'top2_diff': 0.6855, 'exact_hit': False, 'slots': {}}, 'story': 'faq', 'api_exception': 0}], [{'recipient_id': '1710982409.10119', 'text': '运单号：9053'}, {'recipient_id': '1710982409.10119', 'text': '请继续说您的运单号'}, {'recipient_id': '1710982409.10119', 'text': '您好，麻烦您提供一下YT+13位数的圆通运单号码，我帮您查看一下'}, {'recipient_id': '1710982409.10119', 'last_message': {'text': '呃，9053尾号你要填要', 'input_channel': 'callassist', 'intent_name': 'inform', 'confidence': 0.2819, 'top2_diff': 0.118, 'exact_hit': False, 'slots': {}}, 'story': 'faq', 'api_exception': 0}], [{'recipient_id': '1710982409.10119', 'text': '运单号：yt'}, {'recipient_id': '1710982409.10119', 'text': '请继续说您的运单号'}, {'recipient_id': '1710982409.10119', 'text': '您好，麻烦您提供一下10、12或YT+13位数的圆通运单号码，我帮您看一下。'}, {'recipient_id': '1710982409.10119', 'last_message': {'text': '嗯yt', 'input_channel': 'callassist', 'intent_name': 'is_ok', 'confidence': 0.7646, 'top2_diff': 0.6841, 'exact_hit': False, 'slots': {}}, 'story': 'faq', 'api_exception': 0}], [{'recipient_id': '1710982409.10119', 'text': '运单号：yt1850'}, {'recipient_id': '1710982409.10119', 'text': '请继续说您的运单号'}, {'recipient_id': '1710982409.10119', 'text': '您好，麻烦您提供一下YT+13位数的圆通运单号码，我帮您查看一下'}, {'recipient_id': '1710982409.10119', 'last_message': {'text': '1850', 'input_channel': 'callassist', 'intent_name': 'inform', 'confidence': 0.6495, 'top2_diff': 0.4494, 'exact_hit': False, 'slots': {}}, 'story': 'faq', 'api_exception': 0}], [{'recipient_id': '1710982409.10119', 'text': '运单号：yt18508507'}, {'recipient_id': '1710982409.10119', 'text': '请继续说您的运单号'}, {'recipient_id': '1710982409.10119', 'text': '您好，麻烦您提供一下10、12或YT+13位数的圆通运单号码，我帮您看一下。'}, {'recipient_id': '1710982409.10119', 'last_message': {'text': '8507', 'input_channel': 'callassist', 'intent_name': 'service_code', 'confidence': 0.4578, 'top2_diff': 0.1035, 'exact_hit': False, 'slots': {}}, 'story': 'faq', 'api_exception': 0}], [{'recipient_id': '1710982409.10119', 'text': '运单号：yt1850850729053'}, {'recipient_id': '1710982409.10119', 'text': '请继续说您的运单号'}, {'recipient_id': '1710982409.10119', 'text': '您好，麻烦您提供一下YT+13位数的圆通运单号码，我帮您查看一下'}, {'recipient_id': '1710982409.10119', 'last_message': {'text': '29053', 'input_channel': 'callassist', 'intent_name': 'service_code', 'confidence': 0.5624, 'top2_diff': 0.2649, 'exact_hit': False, 'slots': {}}, 'story': 'faq', 'api_exception': 0}], [{'recipient_id': '1710982409.10119', 'text': '请继续说您的运单号'}, {'recipient_id': '1710982409.10119', 'text': '您好，麻烦您提供一下10、12或YT+13位数的圆通运单号码，我帮您看一下。'}, {'recipient_id': '1710982409.10119', 'last_message': {'text': '。对我以后我我不允许他', 'input_channel': 'callassist', 'intent_name': 'angry', 'confidence': 0.9239, 'top2_diff': 0.9148, 'exact_hit': False, 'slots': {}}, 'story': 'faq', 'api_exception': 0}], [{'recipient_id': '1710982409.10119', 'text': '请继续说您的运单号'}, {'recipient_id': '1710982409.10119', 'text': '您好，麻烦您提供一下YT+13位数的圆通运单号码，我帮您查看一下'}, {'recipient_id': '1710982409.10119', 'last_message': {'text': '嗯，我看看啊嗯', 'input_channel': 'callassist', 'intent_name': 'is_ok', 'confidence': 0.6018, 'top2_diff': 0.546, 'exact_hit': False, 'slots': {}}, 'story': 'faq', 'api_exception': 0}], [{'recipient_id': '1710982409.10119', 'text': '请继续说您的运单号'}, {'recipient_id': '1710982409.10119', 'text': '您好，麻烦您提供一下10、12或YT+13位数的圆通运单号码，我帮您看一下。'}, {'recipient_id': '1710982409.10119', 'last_message': {'text': '一个两个', 'input_channel': 'callassist', 'intent_name': 'package_lack', 'confidence': 0.9279, 'top2_diff': 0.9226, 'exact_hit': False, 'slots': {}}, 'story': 'faq', 'api_exception': 0}], [{'recipient_id': '1710982409.10119', 'text': '请继续说您的运单号'}, {'recipient_id': '1710982409.10119', 'text': '您好，麻烦您提供一下YT+13位数的圆通运单号码，我帮您查看一下'}, {'recipient_id': '1710982409.10119', 'last_message': {'text': '嗯', 'input_channel': 'callassist', 'intent_name': 'is_ok', 'confidence': 0.8531, 'top2_diff': 0.8315, 'exact_hit': False, 'slots': {}}, 'story': 'faq', 'api_exception': 0}], [{'recipient_id': '1710982409.10119', 'text': '请继续说您的运单号'}, {'recipient_id': '1710982409.10119', 'text': '您好，麻烦您提供一下10、12或YT+13位数的圆通运单号码，我帮您看一下。'}, {'recipient_id': '1710982409.10119', 'last_message': {'text': '两个', 'input_channel': 'callassist', 'intent_name': 'package_lack', 'confidence': 0.9602, 'top2_diff': 0.9585, 'exact_hit': False, 'slots': {}}, 'story': 'faq', 'api_exception': 0}], [{'recipient_id': '1710982409.10119', 'text': '请继续说您的运单号'}, {'recipient_id': '1710982409.10119', 'text': '您好，麻烦您提供一下YT+13位数的圆通运单号码，我帮您查看一下'}, {'recipient_id': '1710982409.10119', 'last_message': {'text': '，3个目前是目前显示是3个', 'input_channel': 'callassist', 'intent_name': 'package_lack', 'confidence': 0.9724, 'top2_diff': 0.9715, 'exact_hit': False, 'slots': {}}, 'story': 'faq', 'api_exception': 0}], [{'recipient_id': '1710982409.10119', 'text': '请继续说您的运单号'}, {'recipient_id': '1710982409.10119', 'text': '您好，麻烦您提供一下10、12或YT+13位数的圆通运单号码，我帮您看一下。'}, {'recipient_id': '1710982409.10119', 'last_message': {'text': '报名单号', 'input_channel': 'callassist', 'intent_name': 'ask_express_id', 'confidence': 0.7147, 'top2_diff': 0.5816, 'exact_hit': False, 'slots': {}}, 'story': 'faq_intent', 'api_exception': 0}], [{'recipient_id': '1710982409.10119', 'text': '请继续说您的运单号'}, {'recipient_id': '1710982409.10119', 'text': '您好，麻烦您提供一下YT+13位数的圆通运单号码，我帮您查看一下'}, {'recipient_id': '1710982409.10119', 'last_message': {'text': '。呃，连单号应该是3个，然后你能不能能不能给我给我备注一下，以后不让放快递柜。因为我我我放快递柜很不方便。我每天开车从大门进来，你就直接我家里要没有人我我所有的快递，我所有的快递原来全部都是给我放家门口的。我丢件儿已也没有丢过件，从来没丢过件就已经丢丢年了。所以说就给我放家门口就可以了', 'input_channel': 'callassist', 'intent_name': 'home_delivery', 'confidence': 0.6598, 'top2_diff': 0.573, 'exact_hit': False, 'slots': {}}, 'story': 'faq', 'api_exception': 0}], [{'recipient_id': '1710982409.10119', 'last_message': {'text': '，我要不在的情况下', 'input_channel': 'callassist', 'intent_name': 'angry', 'confidence': 0.1032, 'top2_diff': 0.0359, 'exact_hit': False, 'slots': {}}, 'story': 'faq', 'api_exception': 0}], [{'recipient_id': '1710982409.10119', 'text': '请继续说您的运单号'}, {'recipient_id': '1710982409.10119', 'text': '您好，麻烦您提供一下10、12或YT+13位数的圆通运单号码，我帮您看一下。'}, {'recipient_id': '1710982409.10119', 'last_message': {'text': '嗯', 'input_channel': 'callassist', 'intent_name': 'is_ok', 'confidence': 0.8531, 'top2_diff': 0.8315, 'exact_hit': False, 'slots': {}}, 'story': 'faq', 'api_exception': 0}], [{'recipient_id': '1710982409.10119', 'text': '请继续说您的运单号'}, {'recipient_id': '1710982409.10119', 'text': '您好，麻烦您提供一下YT+13位数的圆通运单号码，我帮您查看一下'}, {'recipient_id': '1710982409.10119', 'last_message': {'text': '过', 'input_channel': 'callassist', 'intent_name': 'provided_phone', 'confidence': 0.7689, 'top2_diff': 0.7276, 'exact_hit': False, 'slots': {}}, 'story': 'faq_intent', 'api_exception': 0}], [{'recipient_id': '1710982409.10119', 'text': '请继续说您的运单号'}, {'recipient_id': '1710982409.10119', 'text': '您好，麻烦您提供一下10、12或YT+13位数的圆通运单号码，我帮您看一下。'}, {'recipient_id': '1710982409.10119', 'last_message': {'text': '好的，谢谢啊', 'input_channel': 'callassist', 'intent_name': 'is_ok_thanks', 'confidence': 0.9418, 'top2_diff': 0.9308, 'exact_hit': False, 'slots': {}}, 'story': 'faq', 'api_exception': 0}], [{'recipient_id': '1710982409.10119', 'text': '请继续说您的运单号'}, {'recipient_id': '1710982409.10119', 'text': '您好，麻烦您提供一下YT+13位数的圆通运单号码，我帮您查看一下'}, {'recipient_id': '1710982409.10119', 'last_message': {'text': '，没有了再见', 'input_channel': 'callassist', 'intent_name': 'no_other_questions', 'confidence': 0.9617, 'top2_diff': 0.9584, 'exact_hit': False, 'slots': {}}, 'story': 'faq', 'api_exception': 0}]]

pprint(tem_ls)

[[{'recipient_id': '1710982409.10119',
   'text': '您好，麻烦您提供一下10、12或YT+13位数的圆通运单号码，我帮您看一下。'},
  {'api_exception': 0,
   'last_message': {'confidence': 0.9349,
                    'exact_hit': False,
                    'input_channel': 'callassist',
                    'intent_name': 'home_delivery',
                    'slots': {},
                    'text': '啊，我在你们圆通好几个快递都给我算成放到快递柜儿了。然后我不允许放快递柜，就要上门送到送上门',
                    'top2_diff': 0.9232},
   'recipient_id': '1710982409.10119',
   'story': 'faq'}],
 [{'recipient_id': '1710982409.10119',
   'text': '您好，麻烦您提供一下YT+13位数的圆通运单号码，我帮您查看一下'},
  {'recipient_id': '1710982409.10119',
   'text': '您好，麻烦您提供一下10、12或YT+13位数的圆通运单号码，我帮您看一下。'},
  {'api_exception': 0,
   'last_message': {'confidence': 0.7744,
                    'exact_hit': False,
                    'input_channel': 'callassist',
                    'intent_name': 'package_lost',
                    'slots': {},
                    'text': '快好几个快递呢。我我我我都要告诉你们，好几个快递都给我放空着',
    